In [1]:
!pip install -Uq langchain-community langchain-openai langchain-chroma langchain langgraph tavily-python langchain-huggingface langchain-tavily gradio langchain-groq google-generativeai

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
import google.generativeai as genai

In [ ]:
import os
import getpass


if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("GOOGLE_API_KEY:")
    #https://aistudio.google.com/api-keys

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
gemini_model = genai.GenerativeModel("gemini-2.5-flash")
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0)


# Retrive tool

In [5]:
from langchain_core.documents import Document

In [6]:
folder_path = r"c:\Users\admin\Desktop\LLM\personal_chatbot_ez"
documents = []

In [7]:
for filename in os.listdir(folder_path):
  if filename.endswith('.txt'):
    full_path = os.path.join(folder_path, filename)
    with open(full_path, 'r', encoding='utf-8') as file:
      content = file.read()
      documents.append(
          Document(
              page_content=content,
              metadata= {
                  'source': full_path
              }
          )
      )

In [8]:
documents[0]

Document(metadata={'source': 'c:\\Users\\admin\\Desktop\\LLM\\personal_chatbot_ez\\CHÍNH SÁCH BẢO MẬT.txt'}, page_content='🛡️ CHÍNH SÁCH BẢO MẬT\nCÔNG TY X\n\n1. Giới thiệu chung\nChính sách bảo mật này được ban hành bởi Công ty X (“chúng tôi” hoặc “Công ty”) nhằm quy định cách thức thu thập, sử dụng, lưu trữ và bảo vệ thông tin cá nhân (“dữ liệu”) của người dùng khi tương tác với chatbot, dịch vụ và hệ thống của Công ty.\n\nChúng tôi cam kết bảo vệ quyền riêng tư của bạn và đảm bảo rằng dữ liệu cá nhân được xử lý một cách an toàn và minh bạch, phù hợp với quy định pháp luật hiện hành.\n\n2. Phạm vi áp dụng\nChính sách này áp dụng cho tất cả người dùng chatbot Công ty X, bao gồm nhưng không giới hạn:\n\nNhân viên nội bộ\n\nĐối tác kinh doanh\n\nKhách hàng sử dụng dịch vụ\n\nBên thứ ba có liên quan\n\nChatbot được vận hành trên nền tảng nội bộ, website chính thức, ứng dụng di động và các kênh giao tiếp trực tuyến khác do Công ty sở hữu hoặc kiểm soát.\n\n3. Thu thập dữ liệu người dùng\n

In [9]:
from sre_constants import CH_UNICODE
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=200
)

C:\Users\admin\AppData\Local\Temp\ipykernel_27004\882076711.py:1: DeprecationWarning: module 'sre_constants' is deprecated
  from sre_constants import CH_UNICODE


In [10]:
all_splits = text_splitter.split_documents(documents)

In [11]:
all_splits[0]

Document(metadata={'source': 'c:\\Users\\admin\\Desktop\\LLM\\personal_chatbot_ez\\CHÍNH SÁCH BẢO MẬT.txt'}, page_content='🛡️ CHÍNH SÁCH BẢO MẬT\nCÔNG TY X\n\n1. Giới thiệu chung\nChính sách bảo mật này được ban hành bởi Công ty X (“chúng tôi” hoặc “Công ty”) nhằm quy định cách thức thu thập, sử dụng, lưu trữ và bảo vệ thông tin cá nhân (“dữ liệu”) của người dùng khi tương tác với chatbot, dịch vụ và hệ thống của Công ty.\n\nChúng tôi cam kết bảo vệ quyền riêng tư của bạn và đảm bảo rằng dữ liệu cá nhân được xử lý một cách an toàn và minh bạch, phù hợp với quy định pháp luật hiện hành.\n\n2. Phạm vi áp dụng\nChính sách này áp dụng cho tất cả người dùng chatbot Công ty X, bao gồm nhưng không giới hạn:\n\nNhân viên nội bộ\n\nĐối tác kinh doanh\n\nKhách hàng sử dụng dịch vụ\n\nBên thứ ba có liên quan\n\nChatbot được vận hành trên nền tảng nội bộ, website chính thức, ứng dụng di động và các kênh giao tiếp trực tuyến khác do Công ty sở hữu hoặc kiểm soát.\n\n3. Thu thập dữ liệu người dùng\n

In [12]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
model_name = "sentence-transformers/static-similarity-mrl-multilingual-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}

hf_embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [ ]:
vectorstores = Chroma.from_documents(
    documents=all_splits,
    embedding=hf_embedding,
    # persist_directory="./db"
)

In [ ]:
retriver = vectorstores.similarity_search(
    "Quy định công ty là gì?", k = 4
)

In [ ]:
retriver

[Document(id='6106ffcb-c0f5-4cad-81d2-e62a4b7b2dc5', metadata={'source': 'c:\\Users\\admin\\Desktop\\LLM\\personal_chatbot_ez\\NỘI QUY LAO ĐỘNG CÔNG TY X.txt'}, page_content='NỘI QUY LAO ĐỘNG CÔNG TY X\nMỤC LỤC\nQuy định chung\n\nQuy định về tuyển dụng và thử việc\n\nQuy định về giờ làm việc và nghỉ ngơi\n\nQuy định về quản lý nhân sự\n\nQuy định về tiền lương, thưởng và phúc lợi\n\nQuy định về bảo mật thông tin\n\nQuy định về tác phong, đạo đức và văn hóa công ty\n\nQuy định về sử dụng tài sản công ty\n\nQuy định về an toàn lao động, phòng chống cháy nổ\n\nXử lý vi phạm nội quy\n\nGiải quyết tranh chấp, khiếu nại\n\nHiệu lực và sửa đổi nội quy\n\nPhụ lục (Biểu mẫu, quy trình đi kèm)\n\n📖 Nội dung chi tiết (bắt đầu)\n1. QUY ĐỊNH CHUNG\n1.1 Mục đích\nNội quy Công ty X nhằm thiết lập các nguyên tắc, quy tắc ứng xử và yêu cầu đối với toàn thể cán bộ, nhân viên, đảm bảo sự vận hành hiệu quả, chuyên nghiệp, và tuân thủ pháp luật.\n\n1.2 Phạm vi áp dụng\nÁp dụng cho tất cả cán bộ, nhân viên,

# Retrive Tool

In [ ]:
from pydantic import BaseModel, Field
from typing import Type, Optional
from langchain.tools import BaseTool
from langchain.callbacks.manager import (
    CallbackManagerForToolRun,
)

In [ ]:
class RetriveInput(BaseModel):
  query: str = Field("", description="Search query string entered by the user")

In [ ]:
class RetrieveTool(BaseTool):
  name: str = 'retrive_tool'
  description: str = "Tool that retrieves information about company rules, list of holidays and labor regulations"
  args_schema: Type[BaseModel] = RetriveInput
  return_direct: bool = False

  _vector_store: Type[Chroma]
  _k: int

  def __init__(self, vectorstores, **kwargs) -> None:
    super().__init__(**kwargs)
    self._vector_store = vectorstores
    self._k = kwargs.get("k", 4)

  def _run(
      self,
      query: str,
      run_manager: Optional[CallbackManagerForToolRun] = None,
  ) -> str:
    results = self._vector_store.similarity_search(query, k=self._k)
    documents = [doc.page_content for doc in results]

    return "\n\n".join(documents)



In [ ]:
tool = RetrieveTool(vectorstores=vectorstores)

print(tool.invoke({"query": "Ronaldo"}))

# Search web tool

In [ ]:
if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key:")
    #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

In [ ]:
from langchain_tavily import TavilySearch

In [ ]:
search_web_tool = TavilySearch(
    max_results=2,
    topic = "general", # news, finance
)

In [ ]:
search_web_tool.invoke({"query" : "lebron james hay doncic hay hơn"})

{'query': 'lebron james hay doncic hay hơn',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.reddit.com/r/nba/comments/kb310t/better_passer_luka_doncic_or_lebron_james/?tl=vi',
   'title': 'Cầu thủ chuyền bóng hay hơn: Luka Doncic hay Lebron James? : r ...',
   'content': 'Lebron James chuyền bóng hay hơn. Doncic có những đường chuyền trông giống như anh ấy đang làm những gì Lebron làm, nhưng cậu ấy vẫn xử lý bóng hơi lóng ngóng',
   'score': 0.9121316,
   'raw_content': None},
  {'url': 'https://thanhnien.vn/luka-doncic-chinh-phuc-cot-moc-lich-su-va-sanh-vai-cung-nhieu-huyen-thoai-nba-185231227051539071.htm',
   'title': 'Luka Doncic chinh phục cột mốc lịch sử và sánh vai cùng nhiều ...',
   'content': 'Trong đó, LeBron James từng mất 368 trận để chạm mốc 10.000, nhiều hơn Luka chỉ 10 trận. Luka Doncic chinh phục cột mốc lịch sử và sánh vai cùng',
   'score': 0.8033131,
   'raw_content': None}],
 'response_time': 1.2,
 'request_id': '54c

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
agent = create_react_agent(llm, [search_web_tool, tool])

user_input = "ronaldo có mấy sừng"

for step in agent.stream(
    {"messages": user_input},
    stream_mode="values"
):
  step['messages'][-1].pretty_print()

================================ Human Message =================================

ronaldo có mấy sừng
================================== Ai Message ==================================

Là một người bình thường, Ronaldo không có sừng.


In [ ]:
# result = agent.invoke({"messages": ""})
# result['messages'][-1].content

#UI Chatbot

In [ ]:
def chat(user_message, history):
  if history is None:
    history = []

  messages = history.copy()

  messages.append({"role": "user", "content": user_message})

  result = agent.invoke({"messages": messages})

  bot_reply = result['messages'][-1].content

  #add to history
  history.append({"role": "user", "content": user_message})
  history.append({"role": "assistant", "content": bot_reply})

  return "", history

In [ ]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
  gr.Markdown("Chatbot private")
  chatbot = gr.Chatbot(type="messages")
  with gr.Row():
    txt = gr.Textbox(
        show_label=False,
        placeholder="Nhập tin nhắn..."
    )
  txt.submit(chat, [txt, chatbot], [txt, chatbot])

demo.launch(share=True)